In [1]:
# By Abdalrahman Mohamed Abdallah 
import cv2 as cv
import math
import time
# remove the following comment if you run the notebook on Colab and use cv2_imshow instead cv.imshow()
#from google.colab.patches import cv2_imshow
# import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

faceProto = "modelNweight/opencv_face_detector.pbtxt"
faceModel = "modelNweight/opencv_face_detector_uint8.pb"

ageProto = "modelNweight/age_deploy.prototxt"
ageModel = "modelNweight/age_net.caffemodel"

genderProto = "modelNweight/gender_deploy.prototxt"
genderModel = "modelNweight/gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

padding = 20

def age_gender_detector(frame):
    # Read frame
    t = time.time()
    frameFace, bboxes = getFaceBox(faceNet, frame)
    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
    return frameFace

In [11]:
def readvideo(name):
    video=cv.VideoCapture(name)
    ret, frame = video.read()
    frame_height, frame_width= frame.shape[:2]
    out = cv.VideoWriter('out.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    print("Processing Video...")
    while video.isOpened() :
        hasFrame,frame=video.read()
        
        if name == 0 :
            if cv.waitKey(1) & 0xFF == ord("q"):
                break 

    
        else:
            if not hasFrame:
                cv.waitKey()
                break
        output = age_gender_detector(frame)
        out.write(output)    
    out.release()
    video.release()
    print("Done processing video")



In [12]:
readvideo(0)

Processing Video...
Gender : Male, conf = 1.000
Age Output : [[0.02363629 0.04220302 0.0142708  0.07196763 0.19408125 0.6411149
  0.01006434 0.00266177]]
Age : (38-43), conf = 0.641
Gender : Male, conf = 1.000
Age Output : [[0.02901795 0.17169009 0.03045633 0.06051423 0.17905414 0.51915807
  0.00735569 0.00275355]]
Age : (38-43), conf = 0.519
Gender : Male, conf = 1.000
Age Output : [[0.04377984 0.6336025  0.03778017 0.04706368 0.09108945 0.14120139
  0.00401645 0.00146653]]
Age : (4-6), conf = 0.634
Gender : Male, conf = 1.000
Age Output : [[0.02943313 0.2232216  0.03608165 0.05119834 0.28293854 0.36870804
  0.00626568 0.00215298]]
Age : (38-43), conf = 0.369
Gender : Male, conf = 1.000
Age Output : [[0.03083125 0.39616793 0.04382205 0.10220677 0.2808982  0.13857916
  0.00549173 0.0020029 ]]
Age : (4-6), conf = 0.396
Gender : Male, conf = 1.000
Age Output : [[3.5288392e-03 6.2758140e-02 5.3744882e-02 3.2305682e-01 4.7706306e-01
  7.8491613e-02 9.8704256e-04 3.6949557e-04]]
Age : (25-3

KeyboardInterrupt: 

In [ ]:
def readImage(name):
    input = cv.imread(name)
    output = age_gender_detector(input)
    cv.imshow( "image", output)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [4]:
readImage("./img/girl1.jpg")

Gender : Female, conf = 0.996
Age Output : [[1.3510839e-05 3.6104934e-04 2.3548430e-01 1.0736169e-01 6.5351510e-01
  2.3237276e-03 7.4197655e-05 8.6640404e-04]]
Age : (25-32), conf = 0.654


Processing Video...


KeyboardInterrupt: 